In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install flask-ngrok
!pip install tensorlayer

     |████████████████████████████████| 368kB 3.5MB/s 
     |████████████████████████████████| 3.3MB 10.9MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
  Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import (Input, Conv2d, BatchNorm2d, Elementwise, SubpixelConv2d, Flatten, Dense)
from tensorlayer.models import Model
import os
import numpy as np
# from IPython.display import HTML
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template, request, redirect
import time
from threading import Thread

In [ ]:
def get_G(input_shape):
    w_init = tf.random_normal_initializer(stddev=0.02)
    g_init = tf.random_normal_initializer(1., 0.02)

    nin = Input(input_shape)
    n = Conv2d(64, (3, 3), (1, 1), act=tf.nn.relu, padding='SAME', W_init=w_init)(nin)
    temp = n

    # B residual blocks
    for i in range(16):
        nn = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
        nn = BatchNorm2d(act=tf.nn.relu, gamma_init=g_init)(nn)
        nn = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(nn)
        nn = BatchNorm2d(gamma_init=g_init)(nn)
        nn = Elementwise(tf.add)([n, nn])
        n = nn

    n = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(gamma_init=g_init)(n)
    n = Elementwise(tf.add)([n, temp])
    # B residual blacks end

    n = Conv2d(256, (3, 3), (1, 1), padding='SAME', W_init=w_init)(n)
    n = SubpixelConv2d(scale=2, n_out_channels=None, act=tf.nn.relu)(n)

    n = Conv2d(256, (3, 3), (1, 1), act=None, padding='SAME', W_init=w_init)(n)
    n = SubpixelConv2d(scale=2, n_out_channels=None, act=tf.nn.relu)(n)

    nn = Conv2d(3, (1, 1), (1, 1), act=tf.nn.tanh, padding='SAME', W_init=w_init)(n)
    G = Model(inputs=nin, outputs=nn)
    return G


In [ ]:
lr_img_path = '/content/drive/My Drive/data/valid_LR'
save_dir = "/content/drive/My Drive/data/results"
checkpoint_dir = "/content/drive/My Drive/data/models"
hr_img_path = '/content/drive/My Drive/data/valid_HR'
imids = 0

In [ ]:
valid_hr_img_list = sorted(tl.files.load_file_list(path=hr_img_path, regx='.*.png', printable=False))
valid_lr_img_list = sorted(tl.files.load_file_list(path=lr_img_path, regx='.*.png', printable=False))
valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, path=lr_img_path, n_threads=32)
valid_hr_imgs = tl.vis.read_images(valid_hr_img_list, path=hr_img_path, n_threads=32)
G1 = get_G([1, None, None, 3])
G1.load_weights(os.path.join(checkpoint_dir, 'g.h5'))
G1.eval()

[TL] read 32 from /content/drive/My Drive/data/valid_LR
[TL] read 64 from /content/drive/My Drive/data/valid_LR
[TL] read 96 from /content/drive/My Drive/data/valid_LR
[TL] read 100 from /content/drive/My Drive/data/valid_LR
[TL] read 32 from /content/drive/My Drive/data/valid_HR
[TL] read 64 from /content/drive/My Drive/data/valid_HR
[TL] read 96 from /content/drive/My Drive/data/valid_HR
[TL] read 100 from /content/drive/My Drive/data/valid_HR
[TL] Input  _inputlayer_1: [1, None, None, 3]
[TL] Conv2d conv2d_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_3: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_2: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise element

In [ ]:
def evaluate():
    global imids,G1
    imid = imids[1:]
    imid = int(imid) - 1
    valid_lr_img = valid_lr_imgs[imid]
    valid_hr_img = valid_hr_imgs[imid]

    valid_lr_img = (valid_lr_img / 127.5) - 1

    valid_lr_img = np.asarray(valid_lr_img, dtype=np.float32)
    valid_lr_img = valid_lr_img[np.newaxis,:,:,:]
    size = [valid_lr_img.shape[1], valid_lr_img.shape[2]]

    out = G1(valid_lr_img).numpy()

    print("LR size: %s /  generated HR size: %s" % (size, out.shape))
    print("[*] save images")
    tl.vis.save_image(out[0], os.path.join(save_dir, 'valid_gen.png'))
    tl.vis.save_image(valid_lr_img[0], os.path.join(save_dir, 'valid_lr.png'))
    tl.vis.save_image(valid_hr_img, os.path.join(save_dir, 'valid_hr.png'))

In [ ]:
from flask import redirect, url_for
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    # return render_template("test.html")
     return """
     <style>
     .loader {
      border: 16px solid #f3f3f3;
      border-radius: 50%;
      border-top: 16px solid blue;
      border-right: 16px solid green;
      border-bottom: 16px solid red;
      border-left: 16px solid yellow;
      width: 120px;
      height: 120px;
      -webkit-animation: spin 2s linear infinite;
      animation: spin 2s linear infinite;
     }

     @-webkit-keyframes spin {
      0% { -webkit-transform: rotate(0deg); }
      100% { -webkit-transform: rotate(360deg); }
     }

     @keyframes spin {
      0% { transform: rotate(0deg); }
      100% { transform: rotate(360deg); }
     }
     </style>

     <center><h1>Welcome to Super-Resolution GAN App</h1>
     </center><center><h3>Please click on any image to generate its super-resolution</h3></center>
     <iframe sandbox="allow-scripts" src="https://drive.google.com/embeddedfolderview?id=1pnVqN61GQ_V2nvQzrquin9dcWF8d7vJ6#grid" style="width:100%; height:450px; border:0;"></iframe>
     <br>
     <br>
     <center>
     <form method="POST">
     <input type ="text" name ="text">
     <input type="submit">
     </form>
     """



@app.route('/', methods=['POST'])
# def disp():
  # return "<h1>Processing</h1>",redirect('/output')

# @app.route('/output')
def my_form_post():
    # thr = Thread(target=disp)
    # thr.start()
    text = request.form['text']
    processed_text = text.upper()
    global imids
    imids = processed_text
    evaluate()
    time.sleep(10)
    # return """<iframe src="https://drive.google.com/embeddedfolderview?id=12vf69Wgjw8P9ja7dEZWIwhkHV-5uTgHS#grid" style="width:100%; height:450px; border:0;"></iframe>"""
    return """
    <center>
    <img src="https://drive.google.com/uc?id=1CTuAYXV3fmwyYpLMjvEkj6N7MqMdJFvz" alt="Generated image" style="max-width: 100%;max-height: 97%;">
    <br>
    <h3>Generated Image</h3>
    <br><br>
    <img src="https://drive.google.com/uc?id=1EFP-dkUCJf_dFp0rkoUKzaKWswRJi5Qh" alt="Low-Resolution image">
    <br>
    <h3>Low Resolution Image(Ground Truth)</h3>
    <br><br>
    <img src="https://drive.google.com/uc?id=1qtx0kuBIwL-1Dcw92K852g5nh5gd-X3d" alt="High-Resolution image" style="max-width: 100%;max-height: 97%;">
    <br>
    <h3>High Resolution Image(Ground Truth)</h3>
    <br><br>
    </center>
    """



app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0f0763c5aa91.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Jun/2020 09:06:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2020 09:06:08] "GET /favicon.ico HTTP/1.1" 404 -
